In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os

import torch
import torch.utils.data
from opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory
import cv2

In [2]:
import sys

In [3]:
sys.argv = ['-f', 'ctdetv2', '--exp_id', 'coco_dla','--num_workers', '0','--batch_size','2', '--arch','dlav6_34'] #cmd = 'ctdet --exp_id coco_dla --batch_size 4 --lr 1.25e-4  --gpus 0 --num_workers 0'
opt = opts().parse()
print(opt.task)
print(opt.num_workers)
print(opt.batch_size)

Fix size testing.
training chunk_sizes: [2]
The output will be saved to  C:\Users\avata\FewShot-CenterNet\src\lib\..\..\exp\ctdetv2\coco_dla
ctdetv2
0
2


In [4]:
torch.manual_seed(opt.seed)
torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test

In [40]:
opt.reg_loss

'l1'

In [5]:
Dataset = get_dataset(opt.dataset, opt.task)
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
logger = Logger(opt)
os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
opt.device = torch.device('cuda' if opt.gpus[0] >= 0 else 'cpu')

heads {'hm': 80, 'wh': 2, 'reg': 2}


In [6]:
model = create_model(opt.arch, opt.heads, opt.head_conv)
optimizer = torch.optim.Adam(model.parameters(), opt.lr)
start_epoch = 0
if opt.load_model != '':
    model, optimizer, start_epoch = load_model(model, opt.load_model, optimizer, opt.resume, opt.lr, opt.lr_step)

In [7]:
Trainer = train_factory[opt.task]
trainer = Trainer(opt, model, optimizer)
trainer.set_device(opt.gpus, opt.chunk_sizes, opt.device)

In [8]:
train_loader = torch.utils.data.DataLoader(
      Dataset(opt, 'train'), 
      batch_size=opt.batch_size, 
      shuffle=True,
      num_workers=opt.num_workers,
      pin_memory=True,
      drop_last=True
  )

==> initializing coco 2017 train data.
loading annotations into memory...
Done (t=15.15s)
creating index...
index created!
Loaded train 118287 samples


In [9]:
img = next(iter(train_loader))

In [41]:
print(img.keys())
print(img['input'].size())

dict_keys(['input', 'hm', 'reg_mask', 'ind', 'wh', 'reg'])
torch.Size([2, 3, 512, 512])


In [23]:
model.to('cuda')
torch.cuda.empty_cache()
outputs = model(img['input'].to('cuda'))

In [29]:
output=outputs[0]
print(output.keys())

dict_keys(['objns', 'hm', 'wh', 'reg'])


In [31]:
objns = output['objns'] 
hm = output['hm']
wh = output['wh']
reg = output['reg']
print(objns.size())
print(hm.size())
print(wh.size())
print(reg.size())

torch.Size([2, 1, 128, 128])
torch.Size([2, 80, 128, 128])
torch.Size([2, 2, 128, 128])
torch.Size([2, 2, 128, 128])


In [57]:
img['reg_mask'][0]

tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)

In [58]:
img['ind'][0]

tensor([ 9296,     0, 10176,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [64]:
wh.size()

torch.Size([2, 2, 128, 128])

In [97]:
print(wh.size())
feat = wh.permute(0,2,3,1).contiguous()
print(feat.size())
feat = feat.view(feat.size(0), -1, feat.size(3))
print(feat.size())
print(feat[0])
dim = feat.size(2)
print('dim: ',dim)
ind  = img['ind'].to('cuda').unsqueeze(2).expand(img['ind'].size(0), img['ind'].size(1), dim)
print(ind)
feat = feat.gather(1,ind)
print(feat.size())

torch.Size([2, 2, 128, 128])
torch.Size([2, 128, 128, 2])
torch.Size([2, 16384, 2])
tensor([[-3.1217e-05, -5.6967e-05],
        [-4.0290e-05, -8.2540e-05],
        [-1.2106e-05, -5.4810e-05],
        ...,
        [ 8.3849e-05, -2.0716e-05],
        [ 6.6743e-05, -9.4267e-06],
        [ 2.9766e-05,  1.8917e-05]], device='cuda:0', grad_fn=<SelectBackward>)
dim:  2
tensor([[[ 9296,  9296],
         [    0,     0],
         [10176, 10176],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [

In [104]:
mask=img['reg_mask'].to('cuda').unsqueeze(2).expand_as(feat).float()

In [105]:
feat[0]*mask

tensor([[[ 1.1007e-03, -5.6498e-04],
         [-0.0000e+00, -0.0000e+00],
         [ 6.9714e-04, -5.0806e-04],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00],
 

In [107]:
img['wh'][0]

tensor([[ 7.4050, 10.6600],
        [ 0.0000,  0.0000],
        [10.4900, 15.5700],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0